thisd is  the template for a 5 day signaler on various indexes. 

In [1]:
import sys
sys.path.append("/workspace/QuantLab")

import pandas as pd
from RQCharts.main import RQCharts
from RQOrders.main import RQOrders
import datetime 
import numpy as np
from backtesting import Backtest
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
## initiate classes and set up data that will be needed
##Multiple symbols will end up getting watched do initiate them here

rqchart = RQCharts()
rqorders = RQOrders()
dict_underlying_price = {}
dict_options_chain = {}
today = datetime.datetime.now().strftime('%Y-%m-%d')
ticker = ["SPY", 'TSLA', 'NVDA', 'QQQ']




#define functions to run

def total_signal(df):
   
    for current_pos in range(0,len(df)):

        # Buy condition
        c1 = df['High'].iloc[current_pos] > df['High'].iloc[current_pos-1]
        c2 = df['High'].iloc[current_pos-1] > df['Low'].iloc[current_pos]
        c3 = df['Low'].iloc[current_pos] > df['High'].iloc[current_pos-2]
        c4 = df['High'].iloc[current_pos-2] > df['Low'].iloc[current_pos-1]
        c5 = df['Low'].iloc[current_pos-1] > df['High'].iloc[current_pos-3]
        c6 = df['High'].iloc[current_pos-3] > df['Low'].iloc[current_pos-2]
        c7 = df['Low'].iloc[current_pos-2] > df['Low'].iloc[current_pos-3]

        if c1 and c2 and c3 and c4 and c5 and c6 and c7:
            return 2
        # Symmetrical conditions for short (sell condition)
        c1 = df['Low'].iloc[current_pos] < df['Low'].iloc[current_pos-1]
        c2 = df['Low'].iloc[current_pos-1] < df['High'].iloc[current_pos]
        c3 = df['High'].iloc[current_pos] < df['Low'].iloc[current_pos-2]
        c4 = df['Low'].iloc[current_pos-2] < df['High'].iloc[current_pos-1]
        c5 = df['High'].iloc[current_pos-1] < df['Low'].iloc[current_pos-3]
        c6 = df['Low'].iloc[current_pos-3] < df['High'].iloc[current_pos-2]
        c7 = df['High'].iloc[current_pos-2] < df['High'].iloc[current_pos-3]

        if c1 and c2 and c3 and c4 and c5 and c6 and c7:
            return 1

def add_pointpos_column(df, signal_column):
    """
    Adds a 'pointpos' column to the DataFrame to indicate the position of support and resistance points.
    
    Parameters:
    df (DataFrame): DataFrame containing the stock data with the specified SR column, 'Low', and 'High' columns.
    sr_column (str): The name of the column to consider for the SR (support/resistance) points.
    
    Returns:
    DataFrame: The original DataFrame with an additional 'pointpos' column.
    """
    def pointpos(row):
        if row[signal_column] == 2:
            return row['Low'] - 1e-4
        elif row[signal_column] == 1:
            return row['High'] + 1e-4
        else:
            return np.nan

    df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
    return df

def plot_candlestick_with_signals(df, start_index, num_rows):
    """
    Plots a candlestick chart with signal points.
    
    Parameters:
    df (DataFrame): DataFrame containing the stock data with 'Open', 'High', 'Low', 'Close', and 'pointpos' columns.
    start_index (int): The starting index for the subset of data to plot.
    num_rows (int): The number of rows of data to plot.
    
    Returns:
    None
    """
    df_subset = df[start_index:start_index + num_rows]
    
    fig = make_subplots(rows=1, cols=1)
    
    fig.add_trace(go.Candlestick(x=df_subset.index,
                                 open=df_subset['Open'],
                                 high=df_subset['High'],
                                 low=df_subset['Low'],
                                 close=df_subset['Close'],
                                 name='Candlesticks'),
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset['pointpos'], mode="markers",
                             marker=dict(size=10, color="MediumPurple", symbol='circle'),
                             name="Entry Points"),
                  row=1, col=1)
    
    fig.update_layout(
        width=1200, 
        height=800, 
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
        showlegend=True,
        legend=dict(
            x=0.01,
            y=0.99,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="white"
            ),
            bgcolor="black",
            bordercolor="gray",
            borderwidth=2
        )
    )
    
    fig.show()



import time
import datetime


def simulate_market():
    start_time = datetime.datetime(2025, 2, 13, 9, 30)  # Start at 9:20 AM
    end_time = datetime.datetime(2025, 2, 13, 15, 55)    # End at 4:30 PM
    print(f"Simulating market for...")
    dict_underlying_price = {}
    current_end_time = start_time  # Initialize current_end_time\
    
    # Loop to simulate market data until the end time of the market
    while current_end_time < end_time:
        # For each second, we simulate the market over 5-minute intervals
        for t in ticker:
            
            df = rqchart.add_equity(t, resolution='5m', 
                                    start_time=start_time.strftime('%Y-%m-%d %H:%M:%S'),
                                    end_time=current_end_time.strftime('%Y-%m-%d %H:%M:%S'))
            dict_underlying_price[f'{t}'] = df  # Save the fetched data for future use
            
            if len(df) < 3:
                print(f"Not enough data for {t}. Skipping...")
                continue

            window =df.tail(4).copy()
            signal = total_signal(window)

            # Process the signal and print if it is found
            if signal == 1:
                print(f"SHORT signal found at {df.index[-1]} for {t}")
                dict_options_chain[f"{t}"] = rqchart.add_option_data(t, delta_away_from_atm=0.30)
            elif signal == 2:
                print(f"LONG signal found at {df.index[-1]} for {t}")
                dict_options_chain[f"{t}"] = rqchart.add_option_data(t, delta_away_from_atm=0.30)

        current_end_time += datetime.timedelta(minutes=5)
            

        # Sleep for 1 second before running the next 5-minute simulation
        time.sleep(1)

simulate_market()


/workspace/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Simulating market for...
Not enough data for SPY. Skipping...
Not enough data for TSLA. Skipping...
Not enough data for NVDA. Skipping...
Not enough data for QQQ. Skipping...
Not enough data for SPY. Skipping...
Not enough data for TSLA. Skipping...
Not enough data for NVDA. Skipping...
Not enough data for QQQ. Skipping...
SHORT signal found at 2025-02-13 09:50:00 for TSLA
SHORT signal found at 2025-02-13 11:35:00 for TSLA
SHORT signal found at 2025-02-13 11:40:00 for TSLA
LONG signal found at 2025-02-13 11:55:00 for NVDA
SHORT signal found at 2025-02-13 12:30:00 for SPY


In [10]:
start_time = datetime.datetime(2025, 2, 12, 9, 30)  # Start at 9:20 AM
end_time = datetime.datetime(2025, 2, 12, 16, 30)    # End at 4:30 PM
df = rqchart.add_equity("QQQ", resolution='5m', 
                                    start_time=start_time.strftime('%Y-%m-%d %H:%M:%S'),
                                    end_time=end_time.strftime('%Y-%m-%d %H:%M:%S'))

df

,Open,High,Low,Close,Volume
Time,,,,,
2025-02-12 09:30:00,522.30,522.78,522.13,522.18,345197.0
2025-02-12 09:35:00,523.25,523.50,522.97,523.04,124494.0
2025-02-12 09:40:00,523.73,523.96,523.60,523.96,108001.0
2025-02-12 09:45:00,524.94,525.08,524.82,524.93,222675.0
2025-02-12 09:50:00,525.26,525.41,524.77,524.89,115031.0
...,...,...,...,...,...
2025-02-12 15:35:00,528.75,528.81,528.67,528.72,16977.0
2025-02-12 15:40:00,528.33,528.46,528.32,528.42,40266.0
2025-02-12 15:45:00,527.96,528.10,527.90,528.00,101443.0
